### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** 08 декабря 2020, 08:30   
**Штраф за опоздание:** -2 балла после 08:30 08 декабря, -4 балла после 08:30 15 декабря, -6 баллов после 08:30 22 декабря, -8 баллов после 08:30 29 декабря.

При отправлении ДЗ указывайте фамилию в названии файла Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:
[ML0220, Задание 3] Фамилия Имя. 


Используйте данный Ipython Notebook при оформлении домашнего задания.

##  Реализуем дерево решений (3 балла)

Допишите недостающие части дерева решений. Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn.
Внимание: если Вас не устраивает предложенная структура хранения дерева, Вы без потери баллов можете сделать свой класс DecisionTreeClassifier, в котором сами полностью воспроизведете алгоритм дерева решений. Обязательно в нем иметь только функции fit, predict

In [181]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

from collections import Counter

In [249]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=5, criterion='gini'):
        """
        criterion -- критерий расщепления. необходимо релизовать три:
        Ошибка классификации, Индекс Джини, Энтропийный критерий
        max_depth -- максимальная глубина дерева
        min_samples_split -- минимальное число объектов в листе, чтобы сделать новый сплит
        """
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.num_class = -1
        # Для последнего задания
        self.feature_importances_ = None
        self.criterion = criterion
        # Структура, которая описывает дерево
        # Представляет словарь, где для  node_id (айдишник узла дерева) храним
        # (тип_узла, айдишник признака сплита, порог сплита) если тип NON_LEAF_TYPE
        # (тип_узла, предсказание класса, вероятность класса) если тип LEAF_TYPE
        # Подразумевается, что у каждого node_id в дереве слева 
        # узел с айди 2 * node_id + 1, а справа 2 * node_id + 2
        self.tree = dict()

    def __div_samples(self, x, y, feature_id, threshold):
        """
        Разделяет объекты на 2 множества
        x -- матрица объектов
        y -- вектор ответов
        feature_id -- айдишник признака, по которому делаем сплит
        threshold -- порог, по которому делаем сплит
        """
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]
    
    def __impurity(self, classes):
        """
        Вычисляет impurity вершины в зависимости от self.criterion
        classes - метки классов всех объектов, пришедших в вершину
        """
        if classes.shape[0] == 0:
            return 0
        
        cnts = Counter(classes)
        probs = np.fromiter(cnts.values(), dtype=float)
        probs /= np.sum(probs)
        if (self.criterion == 'gini'):
            impurity = 1 - np.sum(probs ** 2)
        elif (self.criterion == 'entropy'):
            impurity = -np.sum(probs * np.log(probs))
        elif (self.criterion == 'classification_error'):
            impurity = 1 - np.max(probs)
        return impurity
    
    def __information_gain(self, classes, lclasses, rclasses):
        """
        Вычисляет прирост информации для данного сплита
        данной вершины
        classes - метки классов всех объектов данной вершины
        lclasses, rclasses - метки классов объектов, уходящих в 
        левую и правую подвершину данной вершины соответственно
        """
        num = classes.shape[0]
        if num == 0:
            return 0
        
        lnum = lclasses.shape[0]
        rnum = rclasses.shape[0]
        information_gain = self.__impurity(classes) - \
                (lnum / num) * self.__impurity(lclasses) - \
                (rnum / num) * self.__impurity(rclasses)
        return information_gain
    
    def __feature_range(self, x, y, feature_id):
        """
        Для данной фичи генерирует ее значения, возможные
        для перебора в качестве threshold (т. е. те, по которым
        можно будет делать сплиты).
        Берем только те значения фичи, на которых меняется значение таргета.      
        """
        feature = x[:, feature_id]
        indices = np.argsort(feature)
        sorted_y = y[indices]
        sorted_feature = feature[indices]
        
        for i in range(1, sorted_y.shape[0]):
            if sorted_y[i - 1] != sorted_y[i]:
                feature_split_value = (sorted_feature[i - 1] + \
                        sorted_feature[i]) / 2
                yield feature_split_value

    def __find_threshold(self, x, y):
        """
        Находим оптимальный признак и порог для сплита
        Здесь используемые разные impurity в зависимости от self.criterion
        """
        # Ваш код здесь
        
        opt_threshold = None
        opt_feature_id = None
        max_information_gain = -np.inf
        
        for feature_id in range(x.shape[1]):
            threshold = None
            information_gain = -np.inf
            
            feature_range = self.__feature_range(x, y, feature_id)
            for thr in feature_range:
                lx, rx, ly, ry = self.__div_samples(x, y, feature_id, thr)
                ig = self.__information_gain(y, ly, ry)
                if ig > information_gain:
                    information_gain = ig
                    threshold = thr
                    
            if information_gain > max_information_gain:
                max_information_gain = information_gain
                opt_threshold = threshold
                opt_feature_id = feature_id
                    
        return opt_feature_id, opt_threshold

    def __fit_node(self, x, y, node_id, depth):
        """
        Делаем новый узел в дереве
        Решаем, терминальный он или нет
        Если нет, то строим левый узел  с айди 2 * node_id + 1
        И правый узел с  айди 2 * node_id + 2
        """
        # Ваш код здесь
        
        if y.shape[0] == 0:
            return
        
        one_class = np.all(y == y[0])
        
        if one_class \
                or depth == self.max_depth \
                or x.shape[0] == self.min_samples_split:
            cnts = Counter(y)
            predict, prob = cnts.most_common(1)[0]
            prob /= y.shape[0]
            self.tree[node_id] = (
                self.__class__.LEAF_TYPE,
                predict,
                prob
            )
        else:
            feature_id, threshold = self.__find_threshold(x, y)
            self.tree[node_id] = (
                self.__class__.NON_LEAF_TYPE,
                feature_id,
                threshold
            )
            
            lx, rx, ly, ry = self.__div_samples(x, y, feature_id, threshold)
            
            ig = ig = self.__information_gain(y, ly, ry)
            self.feature_importances_[feature_id] += ig;
            
            self.__fit_node(lx, ly, 2 * node_id + 1, depth + 1)
            self.__fit_node(rx, ry, 2 * node_id + 2, depth + 1)
    
    def fit(self, x, y):
        """
        Рекурсивно строим дерево решений
        Начинаем с корня node_id 0
        """
        self.feature_importances_ = {f_id: 0 for f_id in range(x.shape[1])}
        
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        """
        Рекурсивно обходим дерево по всем узлам,
        пока не дойдем до терминального
        """
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]
        
    def predict(self, X):
        """
        Вызывает predict для всех объектов из матрицы X
        """
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)
    
    def get_feature_importance(self):
        """
        Возвращает важность признаков
        """
        # Ваш код здесь
        
        return self.feature_importances_

In [232]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [233]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

In [234]:
clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)
accuracy_score(y_pred=clf.predict(X_test), y_true=y_test)
accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test)

1.0

## Ускоряем дерево решений (2 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine. 
Для этого используем numpy.

In [235]:
%time clf.fit(X_train, y_train)

CPU times: user 1.85 ms, sys: 3 µs, total: 1.85 ms
Wall time: 2.11 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [236]:
%time my_clf.fit(X_train, y_train)

CPU times: user 227 ms, sys: 13 µs, total: 227 ms
Wall time: 226 ms


## Боевое применение (3 балла)

На практике Вы познакомились с датасетом Speed Dating Data. В нем каждая пара в быстрых свиданиях характеризуется определенным набором признаков. Задача -- предсказать, произойдет ли матч пары (колонка match). 

Пример работы с датасетом можете найти в практике пункт 2
https://github.com/VVVikulin/ml1.sphere/blob/master/2019-09/lecture_06/pract-trees.ipynb

Данные и описания колонок лежат тут
https://cloud.mail.ru/public/8nHV/p6J7wY1y1/speed-dating-experiment/

Скачайте датасет, обработайте данные, как показано на семинаре или своим собственным способом. Обучите дерево классифкации. В качестве таргета возьмите колонку 'match'. Постарайтесь хорошо обработать признаки, чтобы выбить максимальную точность. Если точность будет близка к случайному гаданию, задание не будет защитано. 


In [255]:
date = pd.read_csv('./data/Speed Dating Data.csv', encoding='latin1')

In [256]:
date = pd.concat([date.iloc[:, 0], date.iloc[:, 2],
                  date.iloc[:,11:28], date.iloc[:,30:36],
                  date.iloc[:,39:43], date.iloc[:,45:68],
                  date.iloc[:,69:75], date.iloc[:,81:92],
                  date.iloc[:,97:102], date.iloc[:,104:108]], axis=1)
date = date.dropna()

date = date.drop(['sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 
                  'gaming', 'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts', 
                  'music', 'shopping', 'yoga', 'field', 'from', 'career'], axis=1)

In [257]:
X = date.copy()
X = X.drop(['match'], axis=1)
y = date['match']

X = X.values
y = y.values

In [259]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)

In [188]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)

In [129]:
%%time
my_clf.fit(X_train, y_train)
accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test)

CPU times: user 5min 15s, sys: 52.4 ms, total: 5min 15s
Wall time: 5min 15s


0.9880749574105622

Разбейте датасет на трейн и валидацию. Подберите на валидации оптимальный критерий  информативности. 
Постройте графики зависимости точности на валидации от глубины дерева, от минимального числа объектов для сплита. 
Какой максимальной точности удалось достигнуть?

In [253]:
def criterion_scores(X, y):
    kf = KFold(n_splits=2)
    mean_scores = {}
    for criterion in ['gini', 'entropy', 'classification_error']:
        scores = []
        for train_index, test_index in kf.split(X):
            my_clf = MyDecisionTreeClassifier(min_samples_split=2, criterion=criterion)
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            my_clf.fit(X_train, y_train)
            f = my_clf.get_feature_importance()
            score = roc_auc_score(y_pred=my_clf.predict(X_test), y_true=y_test)
            scores.append(score)
        mean_scores[criterion] = np.mean(scores)
    return mean_scores

## Находим самые важные признаки (2 балла)



По построенному дереву  легко понять, какие признаки лучше всего помогли решить задачу. Часто это бывает нужно  не только  для сокращения размерности в данных, но и для лучшего понимания прикладной задачи. Например, Вы хотите понять, какие признаки стоит еще конструировать -- для этого нужно понимать, какие из текущих лучше всего работают в дереве. 

Самый простой метод -- посчитать число сплитов, где использовался данные признак. Это не лучший вариант, так как по признаку который принимает всего 2 значения, но который почти точно разделяет выборку, число сплитов будет очень 1, но при этом признак сам очень хороший. 
В этом задании предлагается для каждого признака считать суммарный gain (в лекции обозначено как Q) при использовании этого признака в сплите. Тогда даже у очень хороших признаков с маленьким число сплитов это значение должно быть довольно высоким.  

Реализовать это довольно просто: создаете словарь номер фичи : суммарный гейн и добавляете в нужную фичу каждый раз, когда используете ее при построении дерева. 

Добавьте функционал, который определяет значения feature importance. Обучите дерево на датасете Speed Dating Data.
Выведите 10 главных фичей по важности.

In [264]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)

In [265]:
my_clf.fit(X_train, y_train)

In [266]:
feature_importance = my_clf.get_feature_importance()
feature_importance

{0: 0,
 1: 0,
 2: 0,
 3: 0,
 4: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 0,
 9: 0,
 10: 0,
 11: 0,
 12: 0,
 13: 0.07663106485292137,
 14: 0,
 15: 0,
 16: 0,
 17: 0,
 18: 0,
 19: 0,
 20: 0,
 21: 0,
 22: 0,
 23: 0,
 24: 0,
 25: 0,
 26: 0,
 27: 0,
 28: 0,
 29: 0,
 30: 0,
 31: 0,
 32: 0,
 33: 0,
 34: 0,
 35: 0,
 36: 0,
 37: 0,
 38: 0,
 39: 0,
 40: 0,
 41: 0,
 42: 0,
 43: 0,
 44: 0,
 45: 0,
 46: 0,
 47: 0,
 48: 0.4784305223497203,
 49: 0,
 50: 0,
 51: 0,
 52: 0,
 53: 0,
 54: 0,
 55: 0,
 56: 0}

## Фидбек (бесценно)

* Какие аспекты обучения деревьев решений Вам показались непонятными? Какое место стоит дополнительно объяснить?

### Ваш ответ здесь

* Здесь Вы можете оставить отзыв о этой домашней работе или о всем курсе.

### ВАШ ОТЗЫВ ЗДЕСЬ

